In [127]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [128]:
from statsmodels.formula.api import ols

## Helper functions!

In [129]:
def run_model(input):
    data = pd.concat([y,input], axis=1)
    formula = f'Withdraw ~ {" + ".join(input.columns)}'
    print(formula)
    model_full = ols(formula, data=data).fit()

    return model_full

## Read the data

In [130]:
# Read the data
data=pd.read_csv('ATM_sample.csv')
data.head()

# Define target variable
y = data['Withdraw']

# Define the input variables
X = data.drop(columns=['Withdraw'])

X.columns

Index(['Shops', 'ATMs', 'Downtown', 'Weekday', 'Center', 'High'], dtype='object')

### Baseline Model for comparison

In [131]:
model = run_model(X)
model.summary()

Withdraw ~ Shops + ATMs + Downtown + Weekday + Center + High


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Withdraw   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 3.656e+05
Date:                Sat, 26 Oct 2024   Prob (F-statistic):               0.00
Time:                        09:57:58   Log-Likelihood:                -51380.
No. Observations:               22000   AIC:                         1.028e+05
Df Residuals:                   21993   BIC:                         1.028e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     10.4284      0.111     94.198      0.000      10.211      10.645
Shops          0.1081      0.001    110.062      0.000       0.106       0.110
ATMs          -1.0096      0.009   -106.982      0.000      -1.028      -0.991
Downtown     -36.1897      0.887    -40.781      0.000     -37.929     -34.450
Weekday       -3.5011      0.037    -93.806      0.000      -3.574      -3.428
Center         7.1931      0.056    129.352      0.000       7.084       7.302
High           0.9566      0.037     26.035      0.000       0.885       1.029
==============================================================================
Omnibus:                    17745.344   Durbin-Watson:                   1.998
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           468940.833
Skew:                           3.781   Prob(JB):                         0.00
Kurtosis:                      24.316   Cond. No.                     4.45e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.45e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Alter the input columns (X) as you see fit

The following example adds combination variables split on downtown.

In [132]:
X_temp = X.drop(columns=['Downtown']).mul(X['Downtown'], axis=0).rename(columns={'Shops': "Shops_DT", 'ATMs': "ATMs_DT", 'Weekday': "Weekday__DT", 'Center': 'Center_DT', 'High': "High_DT"})  

In [133]:
X = pd.concat([X_temp,X], axis=1)
X.head()

,Shops_DT,ATMs_DT,Weekday__DT,Center_DT,High_DT,Shops,ATMs,Downtown,Weekday,Center,High
0,1018,10,0,0,0,1018,10,1,0,0,0
1,974,10,1,0,0,974,10,1,1,0,0
2,0,0,0,0,0,96,2,0,0,0,1
3,958,9,1,0,1,958,9,1,1,0,1
4,0,0,0,0,0,103,4,0,1,0,1


## Run the model with ALL the input variables

(optional step)
I like to do this so we can see the p value and potentially exclude some variables off the bat. 
This also gives us a base model to work with.

In [134]:
model = run_model(X)
print(model.summary()) 

Withdraw ~ Shops_DT + ATMs_DT + Weekday__DT + Center_DT + High_DT + Shops + ATMs + Downtown + Weekday + Center + High
                            OLS Regression Results                            
Dep. Variable:               Withdraw   R-squared:                       0.991
Model:                            OLS   Adj. R-squared:                  0.991
Method:                 Least Squares   F-statistic:                 2.257e+05
Date:                Sat, 26 Oct 2024   Prob (F-statistic):               0.00
Time:                        09:57:58   Log-Likelihood:                -50030.
No. Observations:               22000   AIC:                         1.001e+05
Df Residuals:                   21988   BIC:                         1.002e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
------------

In [136]:
import itertools
import time

# Initialize lists to store metrics
aic = []
bic = []
p = []

# Start time measurement
start_time = time.time()

count = 1

# Null model with just the intercept
formula = 'Withdraw ~ 1'
data = pd.concat([y,X], axis=1)
model = ols(formula, data=data).fit()
aic.append(model.aic)
bic.append(model.bic)
p.append(1)

# Initialize best metrics with the null model
best_aic = model.aic
best_bic = model.bic
best_model_aic = formula
best_model_bic = formula
best_p_aic = 1
best_p_bic = 1

# Iterate over all combinations of predictors
for i in range(1, len(X.columns) + 1):
    for combo in itertools.combinations(X.columns, i):
        # Create the model formula
        formula = f'Withdraw ~ {" + ".join(combo)}'
        model = ols(formula, data=data).fit()
        
        # Store the AIC, BIC, and number of parameters using append
        aic.append(model.aic)
        bic.append(model.bic)
        p.append(i + 1)
        
        # Update best AIC model if current model is better
        if model.aic < best_aic:
            best_model_aic = formula
            best_aic = model.aic
            best_p_aic = i + 1
        
        # Update best BIC model if current model is better
        if model.bic < best_bic:
            best_model_bic = formula
            best_bic = model.bic
            best_p_bic = i + 1

        count += 1

end_time = time.time()
elapsed_time = end_time - start_time

# Output the best model
print(f"Best subset selection took {count} iterations and {elapsed_time:.2f} seconds")
print(f"Best model based on AIC={best_aic:.2f} with {best_p_aic} parameters (incl. intercept):\n {best_model_aic}")
print(f"Best model based on BIC={best_bic:.2f} with {best_p_bic} parameters (incl. intercept):\n {best_model_bic}")